# Základy administrace SQL Serveru
V tomto notebooku následují tyto ukázky:
- Zálohování databáze v **SIMPLE** recovery modelu
- Zálohování databáze ve **FULL** recovery modelu
- Kontrola konzistence databáze příkazem **DBCC CHECKDB**
- Kontrola fragmentace indexů pomocí **sys.dm_db_index_physical_stats**

*Pozn.: Skripty v tomto notebooku lze spustit s tím, že před jejich spuštěním je nutno upravit názvy databází a cesty.*

## 1A - Zálohování databáze v **SIMPLE** recovery modelu
- jednou se nastaví databázová vlastnost **RECOVERY** na **SIMPLE** následujícím příkazem

In [ ]:
USE master
go

ALTER DATABASE AdventureWorks SET RECOVERY SIMPLE
GO

... po provedení předchozího nastavení (*Pozn.: Je dobré toto nastavení provést hned po vytvoření databáze, pokud ji nechceme zálohovat plným postupem*) se periodicky (např. jednou denně) spouští příkaz **BACKUP DATABASE**

In [ ]:
BACKUP DATABASE AdventureWorks TO DISK = 'Z:\MojeZalohy\AW.bak'
WITH
CHECKSUM
, COMPRESSION
, INIT

Uvedený příkaz provede každý den zálohu do stejného souboru:
- díky klíčovému slovu **INIT** bude v souboru vždy poslední provedená záloha
- díky klíčovému slovu **CHECKSUM** může SQL Server při kontrole čitelnosti zálohy přepočítat a porovnat kontrolní součet
- díky klíčovému slovu **COMPRESSION** bude soubor se zálohou menší (klidně 5x) a záloha se vykoná výrazně rychleji; cenou za tuto rychlost je zvýšení zátěže CPU o 5 až 10 %

### Kontrola čitelnosti zálohy
Kontrolu čitelnosti zálohy lze spouštět pokaždé jako součást příkazu BACKUP jako druhý příkaz.

In [ ]:
RESTORE VERIFYONLY FROM DISK = 'Z:\MojeZalohy\AW.bak' WITH CHECKSUM

### Retence záloh

Předchozí příklad uchovává vždy jen poslední zálohu. Pro možnost držet zálohy po delší čas, se dá tento příkaz zmodifikovat tak, že se každému souboru zálohy vypočítá jeho jméno založené na čase.

In [ ]:
DECLARE @datum date = SYSDATETIME()
, @cesta nvarchar(200) = N'Z:\MojeZalohy\AW'

SET @cesta += CONVERT(char(8), @datum, 112) + '.bak'
-- PRINT @cesta

BACKUP DATABASE AdventureWorks TO DISK = @cesta
WITH
CHECKSUM
, COMPRESSION
, INIT

## 1B - Zálohování databáze ve **FULL** recovery modelu
- jednou se nastaví databázová vlastnost **RECOVERY** na **FULL** následujícím příkazem

In [ ]:
USE master
go

ALTER DATABASE AdventureWorks SET RECOVERY FULL
GO

... po provedení předchozího nastavení (*Pozn.: Je dobré toto nastavení provést hned po vytvoření databáze, pokud ji chceme zálohovat plným postupem*) se periodicky (např. jednou denně) spouští příkaz **BACKUP DATABASE**

In [ ]:
BACKUP DATABASE AdventureWorks TO DISK = 'Z:\MojeZalohy\AW.bak'
WITH
CHECKSUM
, COMPRESSION
, INIT

Předchozí příkaz reinicializuje soubor, do kterého se pak budou přidávat zálohy transakčního logu, např. jednou za hodinu, viz následující příkaz:

In [ ]:
BACKUP LOG AdventureWorks TO DISK = 'Z:\MojeZalohy\AW.bak'
WITH
CHECKSUM
, COMPRESSION
, NOINIT

Povšimněme si rozdílů:
- místo klíčového slova **DATABASE** je v příkazu **BACKUP** slovo **LOG**
- místo klíčového slova **INIT** je použito klíčové slovo **NOINIT**, což znamená, že zálohy transakčního logu se budou **přidávat** do stávajícího souboru *.bak*. Pro kontrolu, co všechno je v souboru *.bak* už umístěno, můžeme spustit následující příkaz:

In [ ]:
RESTORE HEADERONLY FROM DISK = 'Z:\MojeZalohy\AW.bak'

## 2 - Monitorování a údržba fragmentace dat
Fragmentaci dat v databázi snadno zjistíme dotazem na DMO **sys.dm_db_index_physical_stats()**. Tato funkce "trpí" na chybu malých čísel, proto je dobré její výsledek trochu odfiltrovat. V následujícím příkladu se zajímáme pouze o fragmentaci větší než 5 % u objektů, které se skládají alespoň ze 100 datových stránek.

In [ ]:
USE AdventureWorks
GO

SELECT
    stat.object_id
    , stat.index_id
    , CONCAT(OBJECT_SCHEMA_NAME(stat.object_id), '.', OBJECT_NAME(stat.object_id)) as full_object_name
    , i.name as index_name
    , avg_fragmentation_in_percent
FROM sys.dm_db_index_physical_stats(DB_ID(), null, null, null, null) as stat
    JOIN sys.indexes as i ON i.object_id = stat.object_id AND i.index_id = stat.index_id
WHERE page_count >= 100 AND avg_fragmentation_in_percent > 5


Předchozí příklad ukazuje monitorování míry fragmentace. Pro její odstranění lze použít např. skript uvedený na této adrese: [GitHub - DropmanCz/sqlscripts](https://github.com/DropmanCz/sqlscripts/blob/master/defragment%20db%20from%20one%20point.sql).

## 3 - Kontrola konzistence databáze
Kontrola konzistence (pokud možno ne náprava zjištěných konzistenčních chyb) se provádí příkazem **DBCC CHECKDB()**. Bohužel, pokud SQL Server není Enterprise edice, tento příkaz vyžaduje databázi v **SINGLE_USER** režimu. V Enterprise edici tento příkaz exkluzivitu přístupu na databázi nepožaduje.

In [ ]:
USE AdventureWorks
GO

DBCC CHECKDB() WITH NO_INFOMSGS

Předcházející volání s použitím **NO_INFOMSGS** by v ideálním případě mělo skončit jen oznámením, že se nic nestalo. Pokud jsou ve výsledku chyby, jedná se o chyby konzistence. Chybový text obsahuje:
- označení poškozené databázové stránky - použije se pozdějí v příkazu **RESTORE DATABASE**
- nabídku, jak stránku "opravit" pomocí **DBCC CHECKDB()** - **NIKDY NEPOUŽÍVAT**